In [ ]:
import itertools
import os

from matplotlib import ticker
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d.art3d import pathpatch_2d_to_3d, Poly3DCollection
import numpy as np
try:
    import open3d as o3d
except ImportError:
    import sysŽ
    print(sys.exc_info())
import pandas as pd
import seaborn as sns

from dosipy.utils.dataloader import load_ear_data
from dosipy.utils.viz import (set_colorblind, fig_config, set_axes_equal,
                              save_fig)
from helpers import (clean_df, export_pcd, export_fields, poynting_vector,
                     export_rect_idx, get_imcolors)

In [ ]:
%config InlineBackend.figure_format = 'retina'

# EM-field distribution

In [ ]:
# input data
polarization = 'te'
f = 26

# load surface data
df = load_ear_data(polarization, f, surface='front')
df = clean_df(df)
xyz = export_pcd(df)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)

# extracting EM data
(Ex, Ey, Ez), (Hx, Hy, Hz) = export_fields(df)
E = np.abs(np.sqrt(Ex ** 2 + Ey ** 2 + Ez ** 2))
H = np.abs(np.sqrt(Hx ** 2 + Hy ** 2 + Hz ** 2))

In [ ]:
fig_config(latex=True, text_size=20, scaler=2)
fig = plt.figure()
ax = plt.axes(projection ='3d')

# point cloud and apd spatial distribution
skip = 1
cs = ax.scatter(xyz[::skip, 2], xyz[::skip, 0], xyz[::skip, 1],
                c=E,
                cmap='viridis',
                s=0.5,
               )
cbar = fig.colorbar(cs, shrink=0.5, pad=-0.1)
tick_locator = ticker.MaxNLocator(nbins=4)
cbar.locator = tick_locator
cbar.update_ticks()
cbar.ax.set_ylabel(r'$|\boldsymbol{E}|$ [V/m]')
cbar.ax.yaxis.labelpad = 10

# additional figure configuration
ax.view_init(10, 90)
ax.set_box_aspect([1, 1, 1])
ax = set_axes_equal(ax)
ax.set_axis_off() 
fig.tight_layout();

# fname = os.path.join('figures', 'e_field_dist')
# save_fig(fig, fname=fname, formats=['png'])

# Parametric projections

In [ ]:
# extract points visible from the plane wave incidence POV (-x direction)
center = pcd.get_center()
edge_length = 20
origin = [-25, 15]
diameter = np.linalg.norm(
    pcd.get_max_bound() - pcd.get_min_bound()
)
radius = 10 ** 5
camera = [center[0] + diameter, center[1], center[2]]
_, mask = pcd.hidden_point_removal(camera, radius)
xyz_zy = xyz[mask]
rect_center = [origin[0] + edge_length / 2, origin[1] + edge_length / 2]
_, rect_idx = export_rect_idx(xyz_zy, rect_center, edge_length, view='zy')

In [ ]:
skip = 1
scaler = 2
fig_config(latex=True, text_size=20, scaler=scaler)
fig = plt.figure()
ax = plt.axes(projection ='3d')

# point cloud
ax.scatter(xyz[::skip, 2], xyz[::skip, 0], xyz[::skip, 1],
           s=0.25, alpha=0.02, color='gray')
ax.scatter(xyz_zy[rect_idx, 2], xyz_zy[rect_idx, 0], xyz_zy[rect_idx, 1],
           s=0.25, alpha=0.2, color='red')
# handle for legend
rect_handle = Rectangle([0,0], 0, 0, color='red', ec='None', alpha=0.8,
                        label='averaging\nsurface')

# averaging surface(s)
rect_pos = xyz[:, 0].max()
edge_length = 20
param_rect = Rectangle(origin, edge_length, edge_length,
                       ec='black', fc='None', lw=2,
                       label='parametric\nsurface')
ax.add_patch(param_rect)
pathpatch_2d_to_3d(param_rect, z=100, zdir='y')

# ax.view_init(15, 70)  # projection a
ax.view_init(0.5, 90)  # projection b
ax.set_box_aspect([1, 1, 1])
ax = set_axes_equal(ax)
ax.set_axis_off()
fig.tight_layout();

# save figure
# fname = os.path.join('figures', 'projection_b_axis_off')
# save_fig(fig, fname=fname, formats=['png'])

# Estimate normals

In [ ]:
# utils
def normals_to_rgb(n):
    """Return RGB color representation of unit vectors.
    
    Ref: Ben-Shabat et al., in proceedings of CVPR 2019, pp. 10104-10112,
         doi: 10.1109/CVPR.2019.01035.
    
    Parameters
    ----------
    n : numpy.ndsarray
        The number of rows correspond to the number of rows of a given
        point cloud, each column corresponds to each component of a
        (normalized) normal vector.
    Returns
    -------
    numpy.ndarray
        corresponding RGB color on the RGB cube
    """
    if n.shape[1] != 3:
        raise ValueError('`n` should be a 3-dimensional vector.')
    n = np.divide(
        n, np.tile(
            np.expand_dims(
                np.sqrt(np.sum(np.square(n), axis=1)), axis=1), [1, 3]))
    rgb = 127.5 + 127.5 * n
    return rgb / 255.

In [ ]:
pcd_rect = o3d.geometry.PointCloud()
pcd_rect.points = o3d.utility.Vector3dVector(np.c_[xyz_zy[rect_idx, 0],
                                                   xyz_zy[rect_idx, 1],
                                                   xyz_zy[rect_idx, 2]])
pcd_rect.estimate_normals()
pcd_rect.orient_normals_consistent_tangent_plane(30)
n_rect = np.asarray(pcd_rect.normals)
rgb_rect = normals_to_rgb(n_rect)

In [ ]:
skip = 1
scaler = 2
fig_config(latex=True, text_size=20, scaler=scaler)
fig = plt.figure()
ax = plt.axes(projection ='3d')

# point cloud
ax.scatter(xyz[::skip, 2], xyz[::skip, 0], xyz[::skip, 1],
           s=0.25, alpha=0.02, color='gray')
ax.scatter(xyz_zy[rect_idx, 2], xyz_zy[rect_idx, 0], xyz_zy[rect_idx, 1],
           s=0.25, alpha=0.1, color='red')
ax.quiver(xyz_zy[rect_idx, 2][::100], xyz_zy[rect_idx, 0][::100], xyz_zy[rect_idx, 1][::100],
          n_rect[:, 2][::100], n_rect[:, 0][::100], n_rect[:, 1][::100],
          length=10, lw=0.5, color='k')

ax.view_init(10, 60)
ax.set_box_aspect([1, 1, 1])
ax = set_axes_equal(ax)
ax.set_axis_off()
fig.tight_layout();

# fname = os.path.join('figures', 'projection_normals_vec')
# save_fig(fig, fname=fname, formats=['png'])

In [ ]:
skip = 1
scaler = 2
fig_config(latex=True, text_size=20, scaler=scaler)
fig = plt.figure()
ax = plt.axes(projection ='3d')

# point cloud
ax.scatter(xyz[::skip, 2], xyz[::skip, 0], xyz[::skip, 1],
           s=0.25, alpha=0.02, color='gray')
ax.scatter(xyz_zy[rect_idx, 2], xyz_zy[rect_idx, 0], xyz_zy[rect_idx, 1],
           s=0.25, alpha=1, color=rgb_rect)

ax.view_init(10, 60)
ax.set_box_aspect([1, 1, 1])
ax = set_axes_equal(ax)
ax.set_axis_off()
fig.tight_layout();

# fname = os.path.join('figures', 'projection_normals_rgb')
# save_fig(fig, fname=fname, formats=['png'])

# RGB cube

In [ ]:
# utils

def draw_unit_cube():
    r = [0, 1]
    X, Y = np.meshgrid(r, r)
    ones = np.ones(4).reshape(2, 2)
    zeros = np.zeros_like(ones)
    ax = plt.axes(projection ='3d')
    ax.plot_surface(X, Y, zeros, lw=3, color='None', edgecolor='k')
    ax.plot_surface(X, Y, zeros, lw=3, color='None', edgecolor='k')
    ax.plot_surface(X, zeros, Y, lw=3, color='None', edgecolor='k')
    ax.plot_surface(X, ones, Y, lw=3, color='None', edgecolor='k')
    ax.plot_surface(ones, X, Y, lw=3, color='None', edgecolor='k')
    ax.plot_surface(zeros, X, Y, lw=3, color='None', edgecolor='k')
    return ax

In [ ]:
pts = np.array(list(itertools.product([0, 1],repeat=3)))
cs = ['black', 'blue', 'green', 'cyan', 'red', 'magenta', 'yellow', 'white']
pairs = pd.DataFrame(data=pts, columns=['x', 'y', 'z'])
pairs['cs'] = cs
pairs

In [ ]:
fig_config(latex=True, text_size=50)
fig = plt.figure(figsize=(6, 6))
ax = draw_unit_cube()
fig.axes.append(ax)
ax.scatter(*pts.T, c=cs, edgecolor='k', depthshade=False, s=1720)
ax.view_init(10, 60)
ax.set_box_aspect([1, 1, 1])
ax = set_axes_equal(ax)
ax.set_axis_off()
fig.tight_layout();

# fname = os.path.join('figures', 'rgb_cube')
# save_fig(fig, fname, formats=['png'])

In [ ]:
fig_config(latex=True, text_size=32)
fig = plt.figure(figsize=(6, 6))
ax = draw_unit_cube()
fig.axes.append(ax)
ax.scatter(pairs['z'], pairs['x'], pairs['y'],
           c=pairs['cs'], edgecolor='k', depthshade=False, s=1720)
ax.set(xlabel='$z$', ylabel='$x$', zlabel='$y$',
       xlim=(-0.1, 1.1), ylim=(-0.1, 1.1), zlim=(-0.1, 1.1),
       xticks=[0, 1], yticks=[0, 1], zticks=[0, 1])
ax.view_init(10, 60)
ax.set_box_aspect([1, 1, 1])
ax = set_axes_equal(ax)
ax.set_axis_off()
fig.tight_layout();

# fname = os.path.join('figures', 'rgb_cube_reversed')
# save_fig(fig, fname, formats=['png'])

# Summary of results

In [ ]:
# utils

color = sns.color_palette(palette='rocket', n_colors=2).as_hex()

def rpd(comp, ref):
    """Relative percentage difference."""
    comp = np.asarray(comp)
    ref = np.asarray(ref)
    val = (comp - ref) / ref * 100
    return val.round(2)

In [ ]:
labels = ['26 GHz', '60 GHz\n(A = 4 cm2)', '60 GHz\n(A = 1 cm2)']
y = np.arange(len(labels))
width = 0.5

ref = [5.3, 6.22, 6.22]
tpd_pol_1 = [5.93, 6.1, 6.64]
rpd_tpd_pol_1 = rpd(tpd_pol_1, ref)
apd_pol_1 = [5.61, 5.83, 6.61]
rpd_apd_pol_1 = rpd(apd_pol_1, ref)
tpd_pol_2 = [4.75, 5.82, 7.48]
rpd_tpd_pol_2 = rpd(tpd_pol_2, ref)
apd_pol_2 = [4.60, 5.71, 7.39]
rpd_apd_pol_2 = rpd(apd_pol_2, ref)

In [ ]:
fig_config(latex=True, text_size=18)
fig, axs = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(15, 2.75))
rects_tpd_1 = axs[0].barh(y - width/2, rpd_tpd_pol_1, width, color=color[0])
rects_apd_1 = axs[0].barh(y + width/2, rpd_apd_pol_1, width, color=color[1])
axs[0].axvline(x=0, lw=1.5, color='k')
rects_tpd_2 = axs[1].barh(y - width/2, rpd_tpd_pol_2, width, color=color[0], label=r'$\text{APD}_1$')
rects_apd_2 = axs[1].barh(y + width/2, rpd_apd_pol_2, width, color=color[1], label=r'$\text{APD}_2$')
axs[1].axvline(x=0, lw=1.5, color='k')

axs[0].bar_label(rects_tpd_1, padding=5)
axs[0].bar_label(rects_apd_1, padding=5)
axs[0].set(xlim=[-10, 15.5],
           yticks=y,
           yticklabels=labels)
axs[0].set_title('Polarization 1', loc='center', fontweight='bold')
axs[0].set_title(r'$\text{APD} < \text{APD}_\text{planar}$', loc='left', fontsize=14)
axs[0].set_title(r'$\text{APD} > \text{APD}_\text{planar}$', loc='right', fontsize=14)
axs[0].tick_params(axis='both', direction="inout")

axs[1].bar_label(rects_tpd_2, padding=5)
axs[1].bar_label(rects_apd_2, padding=5)
axs[1].set(xlim=[-21, 27],
           yticks=y,
           yticklabels=labels)
axs[1].set_title('Polarization 2', loc='center', fontweight='bold')
axs[1].set_title(r'$\text{APD} < \text{APD}_\text{planar}$', loc='left', fontsize=14)
axs[1].set_title(r'$\text{APD} > \text{APD}_\text{planar}$', loc='right', fontsize=14)
axs[1].tick_params(axis='both', direction="inout")

fig.legend(bbox_to_anchor=(1.1, 0.7))
fig.text(0.48, -0.02, s='relative \% difference')
fig.tight_layout()

# fname = os.path.join('figures', 'visual_summary')
# save_fig(fig, fname=fname, formats=['png'])